# Árboles de decisión
## 1. Importación de librerías, establecimiento de directorio de trabajo y carga de datos

In [1]:
# librerías básicas 
import os
import pandas as pd  
import numpy as np  
import warnings  
warnings.filterwarnings('ignore')
  
# para la construcción del modelo 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# algoritmo
from sklearn.tree import DecisionTreeRegressor

# para obtener métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

# para guardar el modelo
import pickle
  
# para visualización
import seaborn as sns  
import matplotlib.pyplot as plt  
from mpl_toolkits.mplot3d import Axes3D  
%matplotlib inline  

os.chdir("C:/Users/aleja/OneDrive - Universidad Politécnica de Madrid/Universidad/Asignaturas/Cuarto/Trabajo de Fin de Grado")

In [2]:
accesos = pd.read_csv("Resultados/Accesos/Accesos_Calidad_Vida.csv", index_col="Unnamed: 0")
accesos.head()

,latitud,longitud,edad,calidad_vida
0,41.621468,2.068474,11,52.885748
1,41.600270,2.085002,12,74.793875
2,41.616524,2.089927,12,79.858657
3,41.618820,2.089480,11,79.858657
4,41.618908,2.089475,12,79.858657


## 2. Construcción de la matriz de características (X) y el vector variable dependiente (y)

In [3]:
X = accesos.iloc[:, :-1].values
y = accesos.iloc[:, -1].values

## 3. División del dataset en set de entrenamiento y de test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## 4. Feature scaling of X (rango: -3 a 3)

In [5]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 5. Construcción y entrenamiento del modelo de regresión sobre el set de entrenamiento

In [14]:
regressor = DecisionTreeRegressor(max_depth=7, max_features='sqrt')
regressor.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=7, max_features='sqrt')

## 6. Predicción de resultados del set de test

In [15]:
y_pred = regressor.predict(X_test)

## 7.  Métricas de rendimiento
Comparamos el set de test con el predicho

In [16]:
# R2score
print('R2score:', r2_score(y_test, y_pred))

# Mean Absolute Error
print('MAE: \t', mean_absolute_error(y_test, y_pred))

# Mean Squared Error
print('MSE: \t', mean_squared_error(y_test, y_pred))

# Root Mean Squared Error
print("RMSE: ", mean_squared_error(y_test, y_pred, squared=False))

R2score: 0.9182718670936422
MAE: 	 2.8933336460662367
MSE: 	 33.4635637236791
RMSE:  5.784769980187553


### 7.1. Para ganar más intuición

In [17]:
np.set_printoptions(precision=2) # redondea a dos decimales

print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

# concatenamos el set predicho y el de test, así podemos ver mejor las diferencias entre los dos

[[34.7  34.51]
 [79.4  79.86]
 [79.37 79.86]
 ...
 [71.58 65.14]
 [79.82 79.86]
 [79.48 79.86]]


### 7.2 Más métricas de rendimiento

In [18]:
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)

print("Media: {:.2f} %".format(accuracies.mean()*100))
print("Desviación estándar: {:.2f} %".format(accuracies.std()*100))

Media: 93.32 %
Desviación estándar: 1.92 %


Vemos que la precision y los errores no son muy buenos, por lo que probaremos otros parámetros para ver si ofrece alguno más óptimo
### 7.3 Grid Search

In [ ]:
parameters = {
    'n_estimators': [95, 100, 105, 165, 170, 175],
    'criterion':['squared_error', 'absolute_error']
}

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = cv,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best r2: {:.2f} %".format(best_accuracy*100))
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Best Parameters:", best_parameters)

## 8. Guardar el mejor modelo

In [ ]:
with open('Modelos/Decission_Tree.pkl', 'wb') as f:
    pickle.dump(regressor, f)

## 9. Ejemplo de predicción

In [ ]:
print(regressor.predict(sc.transform([['425060.955125', '4.606138e+06', '22']]))) 